In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from Word2VecTransformer import Embedding_Word2Vec
from FastTextTransformer import FastTextTransformer
from sklearn.ensemble import RandomForestClassifier
from threading import Thread
from ClearTransformData import Cleardataset
import pandas as pd
import time
import pickle
import os

In [ ]:
model8 = [("fastText",FastTextTransformer(inputFile=Input,ft_home=ft_home,size=300)) ,("clfSVM",LinearSVC())]
param8={"fastText__model":["cbow","skipgram"],"clfSVM__C": [0.1, 1, 10, 100, 1000]}
model11 = [("fastText",FastTextTransformer(inputFile=Input,ft_home=ft_home,size=300)),("clfrandomForest",RandomForestClassifier())]
param11={"fastText__model":["cbow","skipgram"],"clfrandomForest__n_estimators":[200,500,700]}


In [ ]:
modelfinal=[(model8,param8),(model11,param11)]

In [ ]:
if __name__=="__main__":
    print("Debut du programme")
    
    #testData=pd.read_csv("../../data/TestData.csv")
    print("Clear data")
    t1=time.time()
    
    if not os.path.exists("../../data/TrainDataClean.csv"):
        trainData=pd.read_csv("../../data/TrainData.csv")
        XtrainClean=Cleardataset().fit(trainData.description).transform(trainData.description)
        trainData.description=XtrainClean
        trainData.to_csv("../../data/TrainDataClean.csv",index=False)
    else:
        XtrainClean=pd.read_csv("../../data/TrainDataClean.csv")

    print("data cleared in:{}".format(time.time()-t1))
   
    path_to_modele=HOME_VAR+"textmining_with_structured_directory/src/data/"
    demiLen=int(len(modelfinal)/2)
    modelList1 = [modelfinal[i] for i in range(demiLen)]
    modelList2 = [modelfinal[i] for i in range((demiLen+1),len(modelfinal))]

   
    Thread1 = BestModelFinder(X=XtrainClean.description,Y=XtrainClean.Labels,ListParamModel=modelList1,path=path_to_modele)
    Thread2 = BestModelFinder(XtrainClean.description,Y=XtrainClean.Labels,ListParamModel=modelList2,path=path_to_modele)

    print("Execute and save modele:")
    Thread1.start()
    Thread2.start()

    Thread1.join()
    Thread2.join()
